In [5]:
from huggingface_hub import login
from datasets import load_dataset
from dotenv import dotenv_values
import os
import torch
import pandas as pd
from peft import PeftConfig
from transformers import AutoTokenizer
from utils import Evaluator, generate_adapters_list

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
HF_TOKEN_WRITE = dotenv_values(".env.base")['HF_TOKEN_WRITE']
login(token=HF_TOKEN_WRITE)


appendix = '3EpochsLast'# '5EpochsBestF1Train' # '5EpochsBestF1Train' # 5EpochsBestF1Train
log_name_training ='llama_3EpochsLast_cl'
training_type = ''#'NoLora' # 'unmasked'

def extract_params_from_file_name(df: pd.DataFrame, training_type:str=''):
    if training_type == 'NoLora':
        df['model_type'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[1]))
        df['training_config'] = df['dataset'].apply(lambda x: str(x.split('adapters_')[1]))
        df['layer'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[3]))
        df['quantization'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[4]))
        df['gradient_accumulation_steps'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[5]))
        df['learning_rate'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[6]))
        df['run_type'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[7]))
    else:        
        df['model_type'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[1]))
        df['training_config'] = df['dataset'].apply(lambda x: str(x.split('adapters_')[1]))
        df['layer'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[3]))
        df['quantization'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[4]))
        df['r'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[5]))
        df['lora_alpha'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[6]))
        df['lora_dropout'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[7]))
        df['gradient_accumulation_steps'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[8]))
        df['learning_rate'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[9]))
        df['run_type'] = df['dataset'].apply(lambda x: str(x.split('/')[1].split('_')[10]))
    return df

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/pietroferrazzi/.cache/huggingface/token
Login successful


In [6]:

datasets_list = generate_adapters_list(log_name_training, appendix=appendix, training_type=training_type)
peft_config = PeftConfig.from_pretrained(datasets_list[0], token = HF_TOKEN_WRITE)
BASE_MODEL_CHECKPOINT = peft_config.base_model_name_or_path
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_CHECKPOINT,token =HF_TOKEN_WRITE)

evaluation_table = pd.DataFrame(columns=['dataset', 'TP', 'FP', 'FN', 'precision', 'recall', 'f1'])

for i, dataset_checkpoint in enumerate(datasets_list):
    print(f"evaluating {dataset_checkpoint}, {i}/{len(datasets_list)}...")
    test_data = load_dataset(dataset_checkpoint, token=HF_TOKEN, split='test')
    eval = Evaluator(test_data, tokenizer)
    eval.extract_FP_FN_TP_TN_token_by_token()
    eval.create_evaluation_table()
    tmp = eval.evaluation_table.copy()
    tmp['dataset'] = dataset_checkpoint
    evaluation_table = pd.concat([evaluation_table, pd.DataFrame([tmp])])#  evaluation_table.con(tmp)
    print(eval.evaluation_table)

print(evaluation_table)
evaluation_table#.to_csv(f'data/evaluation_table{appendix}.csv', index=False)
evaluation_table = extract_params_from_file_name(evaluation_table, training_type=training_type)
evaluation_table.to_csv(f'data/evaluation_table{training_type}_{appendix}.csv', index=False)
print(f'SAVED TO data/evaluation_table{training_type}_{appendix}.csv')

/Users/pietroferrazzi/Desktop/dottorato/LS-LLaMA-E3C/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


evaluating ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.layer1_NoQuant_64_32_0.01_4_0.0002_3EpochsLast, 0/36...


Map: 100%|██████████| 681/681 [00:00<00:00, 9569.04 examples/s]
/var/folders/9q/smpv54vn7238f0k83jbngk0c0000gn/T/ipykernel_27493/2274005840.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  evaluation_table = pd.concat([evaluation_table, pd.DataFrame([tmp])])#  evaluation_table.con(tmp)


{'TP': 2559, 'FP': 2328, 'FN': 8307, 'precision': 0.523634131368938, 'recall': 0.23550524572059636, 'f1': 0.3248904970481813}
evaluating ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.layer1_NoQuant_16_64_0.01_4_0.0002_3EpochsLast, 1/36...


Map: 100%|██████████| 681/681 [00:00<00:00, 11265.54 examples/s]


...

{'TP': 3170, 'FP': 2644, 'FN': 7696, 'precision': 0.5452356381148951, 'recall': 0.2917356893060924, 'f1': 0.3800959232613909}
evaluating ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.layer1_NoQuant_64_64_0.01_8_0.0002_3EpochsLast, 34/36...


Map: 100%|██████████| 681/681 [00:00<00:00, 11327.73 examples/s]


{'TP': 2539, 'FP': 2312, 'FN': 8327, 'precision': 0.523397237682952, 'recall': 0.23366464200257683, 'f1': 0.3230896481516829}
evaluating ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.layer1_NoQuant_32_32_0.01_2_0.0002_3EpochsLast, 35/36...


Map: 100%|██████████| 681/681 [00:00<00:00, 11109.51 examples/s]


{'TP': 273, 'FP': 301, 'FN': 10593, 'precision': 0.47560975609756095, 'recall': 0.025124240750966316, 'f1': 0.04772727272727273}
                                             dataset     TP    FP     FN  \
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...   2559  2328   8307   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...   2594  2364   8272   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...   1652  1571   9214   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...   2879  1823   7987   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...   3216  2140   7650   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...    242   274  10624   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...   2364  2143   8502   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...  10593  8934    273   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...    282   288  10584   
0  ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.la...    346   419  10520   
0  ferrazzipietro/LS_Llama-2-7b-hf_

In [7]:
best_data_checkpoint = "ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.layer1_NoQuant_64_64_0.01_2_0.0002_3EpochsLast"
                       "ferrazzipietro/LS_Llama-2-7b-hf_adapters_en.layer1_NoQuant_16_64_0.05_4_0.0002_3EpochsLast_clent"
                        " "
best_data = load_dataset(best_data_checkpoint, token=HF_TOKEN, split='test')

In [9]:
best_data[0]

{'sentence': 'The results of the PCR were positive for RNA specific to SARS-CoV-2.',
 'entities': [{'id': '9470',
   'offsets': [19, 22],
   'role': '',
   'semantic_type_id': '',
   'text': 'PCR',
   'type': 'EVENT'},
  {'id': '9485',
   'offsets': [57, 67],
   'role': '',
   'semantic_type_id': '',
   'text': 'SARS-CoV-2',
   'type': 'EVENT'},
  {'id': '10107',
   'offsets': [57, 67],
   'role': '',
   'semantic_type_id': 'C5400365',
   'text': 'SARS-CoV-2',
   'type': 'CLINENTITY'},
  {'id': '10347',
   'offsets': [28, 36],
   'role': '',
   'semantic_type_id': '',
   'text': 'positive',
   'type': 'RML'}],
 'original_text': 'A 43-year-old non-diabetic Indian male reported to our outpatient department with chief complaints of cough with expectoration, chest pain, reduced appetite, fever with chills, and night sweats for two weeks. He also complained of breathlessness on exertion and had two episodes of blood in his sputum. The patient explained that the cough was continuous and was 